In [7]:
import pandas as pd

df = pd.read_csv('amlo_clasify_chatpgt2.csv')
#df = df.head(50)
#df.reset_index(inplace=True)
df

,Texto,Clasificacion,new_classification
0,Buenos días y feliz año. Deseamos que nos vay...,saludos,Support
1,Vamos a iniciar la semana como lo hacemos siem...,NaN,Health
2,También vamos a informar este primer día hábil...,apoyo,Support
3,Y ahora se va a informar porque se está optand...,apoyo,Support
4,En esta etapa se combinan los bancos particula...,apoyo,Support
...,...,...,...
21760,Y el presidente Modi es el presidente mejor ca...,NaN,NaN
21761,"¿La tienes ahí? Ponla, sí, porque no le… Por e...",NaN,NaN
21762,"Una vez, una vez le llegué por medio punto, o ...",NaN,NaN
21763,"Y hay fotos, ¿no? En la India usan esto. Pero,...",NaN,NaN


In [8]:
import os
with open(r'C:\Users\claud\Documents\Descargas\open_ai_key.txt') as f:
    os.environ["OPENAI_API_KEY"] = f.readline()

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name= "gpt-3.5-turbo")


from langchain.schema.messages import HumanMessage, SystemMessage
prompt='''

You are a virtual assistant, who only knows how to classify texts in Spanish and responds only in Spanish, he cannot respond in English.

The classifications that you should take into account are:
Security: talk about security in Mexico
History: facts from the history of Mexico or the world, no comments.
Economy: Refers to the economic situation in Mexico.
Foreign: Talk about how other countries get involved with Mexico or how Mexico impacts abroad, also about migration.
Opinion: People's personal opinion on any topic, mostly opinions about history in the actual society.
Health: Health situation in Mexico.
Support: social and financial support to communities, older adults, indigenous people, etc.
Corruption: About the wrongdoings of other groups or organizations.
Opposition: Opposed to certain ideas or thoughts, even opposition or badmouthing about a person, organization or Politics People.
Construction: Infrastructure built or purchased for the betterment of Mexico.


The answer should only be one word in Spanish for each sentence, not English.


Given you are the virtual assistant classify the following 10 sentences enclosed by quotation marks and separated with commas.
'''

In [9]:
result = []
indice = 0
for i in range(0, len(df), 10):
    result.append('","'.join(df['Texto'].iloc[i:i+10]))
    result[indice] = '"' + result[indice]
    result[indice] = result[indice] + '"'
    indice+=1

In [19]:
inicio = df["new_classification"].index.get_loc(df['new_classification'].last_valid_index())
inicio = (inicio + 1) //10
inicio
fin = len(df) //10
fin

2176

In [5]:
def clasificate_csv(text):

        response = llm.invoke(
                [SystemMessage(content=prompt), HumanMessage(content=text)]
            )
        return response.content

inicio = df["new_classification"].index.get_loc(df['new_classification'].last_valid_index())
inicio = (inicio + 1) //10 
fin = len(df) //10

for i in range(inicio,fin):
    resultado_de_la_clasificacion = clasificate_csv(result[i])
    print(resultado_de_la_clasificacion)
    print(len(resultado_de_la_clasificacion))
    print(type(resultado_de_la_clasificacion))
    df.iloc[i*10:(i+1)*10, 2] = resultado_de_la_clasificacion.split(", ")
    df.to_csv('amlo_clasify_chatpgt2.csv', index=False)
    print("Fin de la asignacion")
    
    
#df["new_clasification"] = df["Texto"].apply(clasificate_csv)


#df.to_csv('amlo_clasify_chatpgt2.csv', index=False)

Support, Health, Support, Support, Support, Support, Support, Security, Construction, Security
94
<class 'str'>
Fin de la asignacion
Seguridad, Opinión, Apoyo, Historia, Historia, Historia, Historia, Historia, Historia, Historia
95
<class 'str'>
Fin de la asignacion
History, History, Opinion, Opinion, Economy, Economy, Support, Construction, Construction, Construction
103
<class 'str'>
Fin de la asignacion
Economy, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opinion
112
<class 'str'>
Fin de la asignacion
Opinion, Opinion, Opinion, Opinion, Opposition, Opinion, Opinion, Opposition, Economy, Security
95
<class 'str'>
Fin de la asignacion
Opinion, Opinion, Foreign, History, Foreign, Opposition, Opposition, Opinion, Opinion, Opinion.
95
<class 'str'>
Fin de la asignacion
Opinion, Opinion, Opinion, Opinion, History, Opinion, History, History, Opinion, Opposition
91
<class 'str'>
Fin de la asignacion
Opinion, Opinion, Opinion, Opinion, Corr

In [11]:
df.head(150)

,Texto,Clasificacion,new_classification
0,Buenos días y feliz año. Deseamos que nos vay...,saludos,Support
1,Vamos a iniciar la semana como lo hacemos siem...,NaN,Health
2,También vamos a informar este primer día hábil...,apoyo,Support
3,Y ahora se va a informar porque se está optand...,apoyo,Support
4,En esta etapa se combinan los bancos particula...,apoyo,Support
...,...,...,...
145,"Cada vez que hay una transformación, hay una n...",opinion,NaN
146,"Entonces, escogimos como 10 reformas muy impor...",seguridad,NaN
147,"Entonces, afortunadamente se llevó a cabo la r...",seguridad,NaN
148,"Doscientos cincuenta, fíjense el cambio; pero...",seguridad,NaN


## 1. Ahorita le mande 50 con el nuevo prompt y si los clasifico de a 10
- Opinion, Support, Support, Support, Support, Support, Support, Security, Support, Opposition
- Security, Support, Opinion, History, History, History, History, Opposition, Opposition, Opposition
- History, History, Opinion, Opinion, Economy, Economy, Support, Construction, Construction, Construction
- Economy, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition.
- Opinion, Opinion, Opposition, Opposition, Opposition, Opinion, Opinion, Opposition, Economy, Security.

### 2. Prompt llamado Bob
- Opinion, Health, Support, Support, Support, Construction, Construction, Security, Support, Security
- Security, History, Support, Opinion, History, History, History, Foreign, Opposition, Opposition
- History, History, Opinion, Opinion, Economy, Economy, Support, Construction, Construction, Construction
- Economy, Opposition, Opposition, Opposition, Corruption, Opposition, Opposition, Corruption, Opposition, Opinion

### 3. Prompt Bob mejorado (de mejor  no tiene nada)
13

- Opinion, Support, Support, Support, Support, Support, Support, Security, Construction, Support
- Seguridad, Apoyo, Apoyo
- Apoyo, Apoyo, Apoyo, Apoyo, Apoyo
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Opposición, Opposición
- History, Opinion, Opinion, Opinion, Economy, Economy, Construction, Construction, Construction, Construction
- Economy, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition
- Opinion, Opinion, Opposition, Opposition, Opposition, Opinion, Opinion, Opposition, Economy, Security


### 4. Prompt sin ser Bob pero mejor 

- Opinion, Support, Support, Support, Support, Construction, Construction, Security, Support, Security
- Seguridad, Apoyo, Seguridad, Historia, Historia, Historia, Historia, Historia, Historia, Historia
- History, Opinion, Opinion, Opinion, Economy, Economy, Support, Construction, Construction, Construction
- Economy, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition
- Opinion, Opinion, Opposition, Opposition, Opposition, Opinion, Opinion, Opposition, Economy, Security

### 5. Prompt anterior pero reiterando que no puede responder en Ingles
- Opinion, Support, Health, Support, Support, Support, Support, Security, Construction, Opposition.
- Seguridad, Apoyo, Seguridad, Historia, Historia, Historia, Historia, Historia, Historia, Historia
- History, Opposition, Opinion, Opinion, Economy, Economy, Construction, Construction, Construction, Construction

99

In [16]:
df.iloc[100]

Texto                 Si ustedes hacen un análisis económico sobre l...
Clasificacion                                                  economia
new_classification                                                  NaN
Name: 100, dtype: object